#  <center><font size="4">Anahita Bagheri Kalhor assignment 3 <br>

In [2]:
pip install protobuf==3.20


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Mohammad\\anaconda3\\Lib\\site-packages\\google\\~upb\\_message.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.3
    Uninstalling protobuf-4.22.3:
      Successfully uninstalled protobuf-4.22.3


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from sklearn.model_selection import train_test_split
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from math import floor
from time import time
import numpy as np
import sklearn.metrics as metrics
import keras.utils.np_utils as nputil
from keras.preprocessing.image import ImageDataGenerator

In [2]:

#load CIfAR-10 Dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()



def pad(x, pad=4):
    return np.pad(x, ((0,0), (pad,pad),(pad,pad),(0,0)), mode='reflect')

def random_crop(x, random_crop_size = (32,32), sync_seed=None):
    np.random.seed(sync_seed)
    w, h = x.shape[1], x.shape[2]
    rangew = (w - random_crop_size[0]) // 2
    rangeh = (h - random_crop_size[1]) // 2
    offsetw = 0 if rangew == 0 else np.random.randint(rangew)
    offseth = 0 if rangeh == 0 else np.random.randint(rangeh)
    return x[:, offsetw:offsetw+random_crop_size[0], offseth:offseth+random_crop_size[1]]

In [3]:
x_train = pad(x_train)
x_test = pad(x_test)

x_train = x_train.astype('float32')
x_train = (x_train - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]
x_test = x_test.astype('float32')
x_test = (x_test - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]
y_train = nputil.to_categorical(y_train)
y_test = nputil.to_categorical(y_test)

In [4]:
def densenet(filters=16):
  growth_rate=12
  compression=0.5
  nr_blocks = 3
  d_rate = 0.2
  repetitions = 6, 12, 24, 16
  
  #composition fuction
  def composition(inp, filters, kernel_size=(3,3), d_rate=0.0):
    x = BatchNormalization()(inp)
    x = ReLU()(x)
    x = Conv2D(filters, 1, strides=1, padding='same')(x)
    out = Dropout(d_rate)(x)
    return out
  
  
  def dense_block(inputs, nr_layers = 4, filters = 16, d_rate = 0, bottleneck = True):
    for _ in range(nr_layers):
      if bottleneck:
        x = composition(inputs, 4*filters)
      else:
        x = inputs
      x = composition(x, filters, (3,3),d_rate)
      inputs = Concatenate(axis = -1)([x, inputs])
    return inputs
  

  def transition_block(inputs, filters, compression = 0.5,d_rate = 0.0):
    filters = floor(filters*compression)
    x = composition(inputs, filters, (1,1), d_rate)
    out = AvgPool2D((2,2), strides=(2,2), padding='same')(x)
    return out
  
  
  input = tf.keras.layers.Input(shape=(32,32,3))
  x = Conv2D(filters, (3,3), use_bias = False)(input)
  x = MaxPool2D((3,3), strides=(2,2), padding='same')(x)
  
  for r in repetitions:
    d = dense_block(x, growth_rate, r, d_rate, False)
    x = transition_block(d, growth_rate, compression, d_rate)
  
  x = GlobalAvgPool2D()(d)
  output = tf.keras.layers.Dense(10,activation='softmax')(x)
  model = Model(input, output)
  return model


In [3]:
generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

generator.fit(x_train, seed=0, augment=True)

test_generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

test_generator.fit(x_test, seed=0, augment=True)



model = densenet()
model.compile(optimizer=('adam'),loss='categorical_crossentropy',metrics=['acc'])
model.summary()
#model.fit(x_train, y_train,
     #        validation_data=(x_test, y_test),
      #       epochs=1,
       #      batch_size=32)

for i in range(0, 4):
    print('i = ' + str(i+1))
    for x_batch, y_batch in generator.flow(x_train, y_train, batch_size=len(x_train)):
        for testx_batch, testy_batch in test_generator.flow(x_test, y_test, batch_size=len(x_test)):
            x_batch = random_crop(x_batch)
            testx_batch = random_crop(testx_batch)
            model.fit(x_batch, y_batch, epochs=1, validation_data=(testx_batch, testy_batch),  batch_size=32)
            break
        break
yPreds = model.predict(random_crop(x_test))
yPred = np.argmax(yPreds, axis=1)
yPred = nputil.to_categorical(yPred)
yTrue = y_test

accuracy = metrics.accuracy_score(yTrue, yPred) * 100
print("Accuracy : ", accuracy)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_106 (Conv2D)            (None, 30, 30, 16)   432         ['input_3[0][0]']                
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 15, 15, 16)  0           ['conv2d_106[0][0]']             
                                                                                                  
 batch_normalization_104 (Batch  (None, 15, 15, 16)  64          ['max_pooling2d_2[0][0]']        
 Normalization)                                                                             

                                                                                                  
 re_lu_110 (ReLU)               (None, 15, 15, 52)   0           ['batch_normalization_110[0][0]']
                                                                                                  
 conv2d_113 (Conv2D)            (None, 15, 15, 6)    318         ['re_lu_110[0][0]']              
                                                                                                  
 dropout_110 (Dropout)          (None, 15, 15, 6)    0           ['conv2d_113[0][0]']             
                                                                                                  
 concatenate_102 (Concatenate)  (None, 15, 15, 58)   0           ['dropout_110[0][0]',            
                                                                  'concatenate_101[0][0]']        
                                                                                                  
 batch_nor

 Normalization)                                                                                   
                                                                                                  
 re_lu_117 (ReLU)               (None, 8, 8, 6)      0           ['batch_normalization_117[0][0]']
                                                                                                  
 conv2d_120 (Conv2D)            (None, 8, 8, 12)     84          ['re_lu_117[0][0]']              
                                                                                                  
 dropout_117 (Dropout)          (None, 8, 8, 12)     0           ['conv2d_120[0][0]']             
                                                                                                  
 concatenate_108 (Concatenate)  (None, 8, 8, 18)     0           ['dropout_117[0][0]',            
                                                                  'average_pooling2d_8[0][0]']    
          

 batch_normalization_124 (Batch  (None, 8, 8, 90)    360         ['concatenate_114[0][0]']        
 Normalization)                                                                                   
                                                                                                  
 re_lu_124 (ReLU)               (None, 8, 8, 90)     0           ['batch_normalization_124[0][0]']
                                                                                                  
 conv2d_127 (Conv2D)            (None, 8, 8, 12)     1092        ['re_lu_124[0][0]']              
                                                                                                  
 dropout_124 (Dropout)          (None, 8, 8, 12)     0           ['conv2d_127[0][0]']             
                                                                                                  
 concatenate_115 (Concatenate)  (None, 8, 8, 102)    0           ['dropout_124[0][0]',            
          

                                                                                                  
 batch_normalization_131 (Batch  (None, 4, 4, 30)    120         ['concatenate_120[0][0]']        
 Normalization)                                                                                   
                                                                                                  
 re_lu_131 (ReLU)               (None, 4, 4, 30)     0           ['batch_normalization_131[0][0]']
                                                                                                  
 conv2d_134 (Conv2D)            (None, 4, 4, 24)     744         ['re_lu_131[0][0]']              
                                                                                                  
 dropout_131 (Dropout)          (None, 4, 4, 24)     0           ['conv2d_134[0][0]']             
                                                                                                  
 concatena

                                                                  'concatenate_126[0][0]']        
                                                                                                  
 batch_normalization_138 (Batch  (None, 4, 4, 198)   792         ['concatenate_127[0][0]']        
 Normalization)                                                                                   
                                                                                                  
 re_lu_138 (ReLU)               (None, 4, 4, 198)    0           ['batch_normalization_138[0][0]']
                                                                                                  
 conv2d_141 (Conv2D)            (None, 4, 4, 24)     4776        ['re_lu_138[0][0]']              
                                                                                                  
 dropout_138 (Dropout)          (None, 4, 4, 24)     0           ['conv2d_141[0][0]']             
          

 concatenate_133 (Concatenate)  (None, 2, 2, 38)     0           ['dropout_144[0][0]',            
                                                                  'concatenate_132[0][0]']        
                                                                                                  
 batch_normalization_145 (Batch  (None, 2, 2, 38)    152         ['concatenate_133[0][0]']        
 Normalization)                                                                                   
                                                                                                  
 re_lu_145 (ReLU)               (None, 2, 2, 38)     0           ['batch_normalization_145[0][0]']
                                                                                                  
 conv2d_148 (Conv2D)            (None, 2, 2, 16)     624         ['re_lu_145[0][0]']              
                                                                                                  
 dropout_1

                                                                                                  
 concatenate_140 (Concatenate)  (None, 2, 2, 150)    0           ['dropout_151[0][0]',            
                                                                  'concatenate_139[0][0]']        
                                                                                                  
 batch_normalization_152 (Batch  (None, 2, 2, 150)   600         ['concatenate_140[0][0]']        
 Normalization)                                                                                   
                                                                                                  
 re_lu_152 (ReLU)               (None, 2, 2, 150)    0           ['batch_normalization_152[0][0]']
                                                                                                  
 conv2d_155 (Conv2D)            (None, 2, 2, 16)     2416        ['re_lu_152[0][0]']              
          